<a href="https://colab.research.google.com/github/MohammadFebriyanto/TrashDetection/blob/master/reyhan_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Library yang akan digunakan
import os
import zipfile
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
from keras.preprocessing import image

# Cek versi dari tensorflow
print(tf.__version__)

2.2.0


Using TensorFlow backend.


In [2]:
# Download file zip berisikan gambar
!wget --no-check-certificate \
  https://github.com/garythung/trashnet/blob/master/data/dataset-resized.zip?raw=true \
  -O /tmp/trashnet.zip

--2020-06-04 13:56:34--  https://github.com/garythung/trashnet/blob/master/data/dataset-resized.zip?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/garythung/trashnet/raw/master/data/dataset-resized.zip [following]
--2020-06-04 13:56:34--  https://github.com/garythung/trashnet/raw/master/data/dataset-resized.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/garythung/trashnet/master/data/dataset-resized.zip [following]
--2020-06-04 13:56:34--  https://raw.githubusercontent.com/garythung/trashnet/master/data/dataset-resized.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... con

In [0]:
# Ekstrak file Zip
local_zip = '/tmp/trashnet.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [4]:
os.listdir('/tmp/dataset-resized')

['glass',
 'validation',
 '.DS_Store',
 'trash',
 'plastic',
 'paper',
 'metal',
 'cardboard',
 'train']

In [0]:
# Membuat Folder Training dan Validation
# os.mkdir(os.path.join('/tmp/dataset-resized/', 'train/'))
# os.mkdir(os.path.join('/tmp/dataset-resized/', 'validation/'))
os.mkdir(os.path.join('/tmp/dataset-resized/validation/', 'cardboard/'))
os.mkdir(os.path.join('/tmp/dataset-resized/validation/', 'glass/'))
os.mkdir(os.path.join('/tmp/dataset-resized/validation/', 'metal/'))
os.mkdir(os.path.join('/tmp/dataset-resized/validation/', 'paper/'))
os.mkdir(os.path.join('/tmp/dataset-resized/validation/', 'plastic/'))
os.mkdir(os.path.join('/tmp/dataset-resized/validation/', 'trash/'))

In [0]:
# Memindahkan file gambar ke dua folder (gambar masih terduplikat)
folder_list = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
for folder in folder_list:
  shutil.move('/tmp/dataset-resized/' + folder, '/tmp/dataset-resized/train')

In [0]:
for folder in folder_list:
  fraction = int(len(os.listdir('/tmp/dataset-resized/train/' + folder + '/'))*0.2) #jumlah 20% data yang akan dipindahkan
  folder_dir = os.listdir('/tmp/dataset-resized/train/' + folder + '/')
  chosen_file = folder_dir[:fraction] #memilih 20% pada urutan awal
  for file in chosen_file:
    shutil.move('/tmp/dataset-resized/train/' + folder + '/' + file, '/tmp/dataset-resized/validation/'+ folder + '/' + file)

In [11]:
# Jumlah file gambar di folder train setelah dipindahkan
print('Pada folder train:')
print('cardboard images: ' + str(len(os.listdir('/tmp/dataset-resized/train/cardboard/'))))
print('glass images: ' + str(len(os.listdir('/tmp/dataset-resized/train/glass/'))))
print('metal images: ' + str(len(os.listdir('/tmp/dataset-resized/train/metal/'))))
print('paper images: ' + str(len(os.listdir('/tmp/dataset-resized/train/paper/'))))
print('plastic images: ' + str(len(os.listdir('/tmp/dataset-resized/train/plastic/'))))
print('trash images: ' + str(len(os.listdir('/tmp/dataset-resized/train/trash/'))))


Pada folder train:
cardboard images: 323
glass images: 401
metal images: 328
paper images: 476
plastic images: 386
trash images: 110


In [12]:
# Jumlah file gambar di folder validation setelah dipindahkan
print('Pada folder validation:')
print('cardboard images: ' + str(len(os.listdir('/tmp/dataset-resized/validation/cardboard/'))))
print('glass images: ' + str(len(os.listdir('/tmp/dataset-resized/validation/glass/'))))
print('metal images: ' + str(len(os.listdir('/tmp/dataset-resized/validation/metal/'))))
print('paper images: ' + str(len(os.listdir('/tmp/dataset-resized/validation/paper/'))))
print('plastic images: ' + str(len(os.listdir('/tmp/dataset-resized/validation/plastic/'))))
print('trash images: ' + str(len(os.listdir('/tmp/dataset-resized/validation/trash/'))))

Pada folder validation:
cardboard images: 80
glass images: 100
metal images: 82
paper images: 118
plastic images: 96
trash images: 27


In [42]:
folder_train = '/tmp/dataset-resized/train/'
train_datagen = ImageDataGenerator(
      rotation_range = 90,
      width_shift_range = 0.2,
      height_shift_range = 0.2,
      rescale = 1./255,
      shear_range = 0.2,
      zoom_range = 0.2,
      fill_mode='nearest')

folder_validation = '/tmp/dataset-resized/validation/'

train_generator = train_datagen.flow_from_directory(
        folder_train, 
        target_size=(150, 150),  
        batch_size=20,
        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
        folder_validation, 
        target_size=(150, 150),  
        batch_size=20,
        class_mode='categorical')

Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [43]:
model = tf.keras.models.Sequential([

tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
tf.keras.layers.MaxPooling2D(2, 2),
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(6, activation='softmax') # menggunakan 3 neuron di akhir dengan fungsi aktivasi softmax sebagai probabilitas
])
model.summary()

model.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['acc'])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 15, 15, 128)     

In [44]:
model.fit(
      train_generator,
      steps_per_epoch=20, 
      epochs=50,
      validation_data=validation_generator, 
      validation_steps=5,  
      verbose=1)

Epoch 1/50
20/20 [==============================] - 13s 627ms/step - loss: 2.0116 - acc: 0.2100 - val_loss: 1.7214 - val_acc: 0.2500
Epoch 2/50
20/20 [==============================] - 12s 622ms/step - loss: 1.7578 - acc: 0.2025 - val_loss: 1.7304 - val_acc: 0.2500
Epoch 3/50
20/20 [==============================] - 12s 623ms/step - loss: 1.7662 - acc: 0.2350 - val_loss: 1.6743 - val_acc: 0.3000
Epoch 4/50
20/20 [==============================] - 12s 606ms/step - loss: 1.7289 - acc: 0.2292 - val_loss: 1.6598 - val_acc: 0.2600
Epoch 5/50
20/20 [==============================] - 12s 602ms/step - loss: 1.6580 - acc: 0.2682 - val_loss: 1.7019 - val_acc: 0.1900
Epoch 6/50
20/20 [==============================] - 12s 620ms/step - loss: 1.6290 - acc: 0.2800 - val_loss: 1.6611 - val_acc: 0.2900
Epoch 7/50
20/20 [==============================] - 12s 619ms/step - loss: 1.6578 - acc: 0.3475 - val_loss: 1.7467 - val_acc: 0.2100
Epoch 8/50
20/20 [==============================] - 12s 620ms/step - 